In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from backtester import backtester
from strat import *
pd.set_option('display.max_rows', 1001)

In [2]:
data = pd.read_csv("data.csv")

In [3]:
ticker_dict = {}

for ticker in data.ticker.unique():
        data_tmp = data.loc[data["ticker"] == ticker]
        data_tmp['ret'] = data_tmp['last'].pct_change()
        data_tmp['log_ret'] = np.log(data_tmp['last']) - np.log(data['last'].shift(1))
        data_tmp['signal'] = data_tmp['log_ret']
        ticker_dict[ticker] = data_tmp.set_index('date')

<ipython-input-3-1b419a303e67>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tmp['ret'] = data_tmp['last'].pct_change()
<ipython-input-3-1b419a303e67>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tmp['log_ret'] = np.log(data_tmp['last']) - np.log(data['last'].shift(1))
<ipython-input-3-1b419a303e67>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [4]:
uni_date = sorted(data.date.unique())

In [5]:
data_price = pd.DataFrame(index = uni_date)

for ticker in data.ticker.unique():
    data_price[ticker] = ticker_dict[ticker]['last']

In [6]:
import statsmodels.tsa.stattools as ts
import statsmodels.api as sm

In [7]:
def conintegrate(df, crit = 0.05):
    df.dropna(axis=1,how='all', inplace = True)
    n = df.shape[1]
    pairs = {}
    tickers = df.columns
    
    for i in range(n):
        for j in range(i+1, n):
            df_price = pd.DataFrame(index = df.index)
            df_price[tickers[i]] = df[tickers[i]]
            df_price[tickers[j]] = df[tickers[j]]
            df_price.dropna(axis = 0, inplace = True)
            try:
                result = sm.tsa.stattools.coint(df_price[tickers[i]], df_price[tickers[j]]) 
                pvalue = result[1] 
                if pvalue < crit:
                    pairs[f'{tickers[i]} & {tickers[j]}'] = pvalue
            except:
                pass
    return pairs

In [8]:
coint_pairs = conintegrate(data_price.iloc[:500])

<ipython-input-7-2ad33ae3deb4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=1,how='all', inplace = True)


In [17]:
df_coint_pairs = pd.DataFrame.from_dict(coint_pairs, orient='index')
df_coint_pairs.columns = ["coint pv"]
df_coint_pairs.to_csv("coint_pairs_500.csv")

In [7]:
coint_pairs = pd.read_csv("coint_pairs_500.csv").set_index("Unnamed: 0").to_dict()['coint pv']

In [10]:
from math import sqrt

In [11]:
results = []
cnt = 0
for pairs in coint_pairs:
    print(cnt)
    ticker1, ticker2 = pairs.split(' & ')
    try:
        sharpe= kalmanTrading(data_price.iloc[500:1500], ticker1, ticker2)
    except:
        sharpe = np.nan
    
    results.append(sharpe)
    cnt += 1

<ipython-input-9-3baf19272bc0>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units long'][0] = 0
<ipython-input-9-3baf19272bc0>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units short'][0] = 0


0.4715238712984126
nan


<ipython-input-9-3baf19272bc0>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units long'][0] = 0
<ipython-input-9-3baf19272bc0>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units short'][0] = 0


1.6683287544259269


<ipython-input-9-3baf19272bc0>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units long'][0] = 0
<ipython-input-9-3baf19272bc0>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units short'][0] = 0


...

<ipython-input-9-3baf19272bc0>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units long'][0] = 0
<ipython-input-9-3baf19272bc0>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units short'][0] = 0


0.9770737146173111


<ipython-input-9-3baf19272bc0>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units long'][0] = 0
<ipython-input-9-3baf19272bc0>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units short'][0] = 0


0.0
1.5658447730107665


<ipython-input-9-3baf19272bc0>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units long'][0] = 0
<ipython-input-9-3baf19272bc0>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units short'][0] = 0


In [16]:
len(results)

1925

In [17]:
coint_pairs_test = {}
cnt = 0
for pair in coint_pairs:
    coint_pairs_test[pair] = results[cnt]
    cnt += 1

In [19]:
df_coint_pairs_test = pd.DataFrame.from_dict(coint_pairs_test, orient='index')
df_coint_pairs_test.columns = ['sharpe']
df_coint_pairs_test.dropna(inplace = True, axis = 0)
df_coint_pairs_test.sort_values(by = ['sharpe'], ascending = False)

In [75]:
coint_pairs_oos = df_coint_pairs_test.loc[df_coint_pairs_test['sharpe'] > 2]
coint_pairs_oos

,sharpe
1333 JT & 1925 JT,2.186514
2002 JT & 2282 JT,2.094354
3101 JT & 6701 JT,2.309897
4004 JT & 5706 JT,2.137615
4183 JT & 7733 JT,2.051517
4543 JT & 6302 JT,2.188670
5631 JT & 6702 JT,2.576582
5631 JT & 8031 JT,2.286998


In [6]:
coint_pairs_oos = pd.read_csv("coint_pairs_oos.csv").set_index("Unnamed: 0")

In [7]:
coint_pairs_oos

,sharpe
Unnamed: 0,
1333 JT & 1925 JT,2.186514
2002 JT & 2282 JT,2.094354
3101 JT & 6701 JT,2.309897
4004 JT & 5706 JT,2.137615
4183 JT & 7733 JT,2.051517
4543 JT & 6302 JT,2.188670
5631 JT & 6702 JT,2.576582
5631 JT & 8031 JT,2.286998


In [8]:
results_oos = []

for pairs in coint_pairs_oos.index:
    print(pairs)
    ticker1, ticker2 = pairs.split(' & ')
    
    try:
        sharpe= kalmanTrading2(data_price.iloc[1200:], ticker1, ticker2)
    except:
        sharpe = np.nan
    
    results_oos.append(sharpe)


1333 JT & 1925 JT


C:\Users\user\OneDrive\Documents\Y-Intercept\Y-Intercept\strat.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units long'][0] = 0
C:\Users\user\OneDrive\Documents\Y-Intercept\Y-Intercept\strat.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units short'][0] = 0


2002 JT & 2282 JT


C:\Users\user\OneDrive\Documents\Y-Intercept\Y-Intercept\strat.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units long'][0] = 0
C:\Users\user\OneDrive\Documents\Y-Intercept\Y-Intercept\strat.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units short'][0] = 0


3101 JT & 6701 JT


...

C:\Users\user\OneDrive\Documents\Y-Intercept\Y-Intercept\strat.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units long'][0] = 0
C:\Users\user\OneDrive\Documents\Y-Intercept\Y-Intercept\strat.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units short'][0] = 0


5631 JT & 6702 JT


C:\Users\user\OneDrive\Documents\Y-Intercept\Y-Intercept\strat.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units long'][0] = 0
C:\Users\user\OneDrive\Documents\Y-Intercept\Y-Intercept\strat.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units short'][0] = 0


5631 JT & 8031 JT


C:\Users\user\OneDrive\Documents\Y-Intercept\Y-Intercept\strat.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units long'][0] = 0
C:\Users\user\OneDrive\Documents\Y-Intercept\Y-Intercept\strat.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['num units short'][0] = 0


In [10]:
results_oos

[0.09414737873328023,
 1.4667744304216774,
 2.3811088835669643,
 1.3824277661544457,
 0.9785019322756111,
 1.0175134182397216,
 2.1338516885629395,
 1.7510264979147971]